In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag_gumbel_softmax import train_gumbel_sgd
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm
import MEC
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [4]:
def nodag_vote_adjacency(R_hat, lam=0.5, delta=1e-6, max_steps=5000, 
                         tau_start=0.2, tau_end=0.2, times=100):
    vote_counter = {}          # 统计每个邻接矩阵出现的次数
    all_results = []           # 保存每次 run 的完整结果，便于回溯

    for t in range(times):
        np.random.seed(t)      # 仅控制 NumPy 随机性（用于 B_init）
        B_init = np.random.randn(*R_hat.shape)

        # 训练一次
        B_final, G_final, info = train_gumbel_sgd(
            Rhat_np=R_hat,
            lam=lam,
            delta=delta,
            max_steps=max_steps,
            tau_start=tau_start,
            tau_end=tau_end,
            B_init=B_init
        )

        # —— 建议：投票前把对角线强制置零，避免“自环”差异影响投票 —— 
        G_key_mat = G_final.copy()
        np.fill_diagonal(G_key_mat, 0)

        # 将矩阵转为可哈希的 key（tuple of ints）
        key = tuple(G_key_mat.flatten().tolist())

        vote_counter[key] = vote_counter.get(key, 0) + 1
        all_results.append((G_final, B_final, info))

    # 选票最多的邻接矩阵（若并列，max 会返回遇到的第一个）
    best_key = max(vote_counter, key=vote_counter.get)
    best_count = vote_counter[best_key]

    # 找到对应的具体结果（取第一个出现的就行）
    for G_final, B_final, info in all_results:
        G_key_mat = G_final.copy()
        np.fill_diagonal(G_key_mat, 0)
        if tuple(G_key_mat.flatten().tolist()) == best_key:
            return (
                G_final, B_final, 
                info["final_loss"], 
                info["final_likelihood"], 
                info["final_penalty"], 
                best_count / times, 
                vote_counter
            )


In [ ]:
for i in range(1, 6):
    X, Y, Z, G_true, CPDAG = generate_scm_data(i, 10000)
    A_true = (np.eye(3) - G_true)
    data = np.array([X, Y, Z]).T
    R_hat = np.cov(data.T)
    B_init = np.random.randn(*R_hat.shape)
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
    best_G, best_B, best_loss, best_likelihood, best_penalty, rate, vote_counter = nodag_vote_adjacency(R_hat = R_hat, times = 100)
    print("SCM: ",i)
    print("likelihood_true = ", likelihood_true)
    print("G_true = \n", G_true)
    print("G_est = \n", best_G)
    # print("Is in MEC: ", MEC.is_in_markov_equiv_class(G_true, best_B))
    print("Final Loss = ", best_loss)
    print("Final penalty = ", best_penalty)
    print("Final likelihood = ", best_likelihood)
    print("rate = ", rate)
    print("")
    

SCM:  1
likelihood_true =  2.995211487185048
G_true = 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  4.494673661500312
Final penalty =  1.500000000000246
Final likelihood =  2.994673661500066
rate =  0.17

SCM:  2
likelihood_true =  3.0179642443567762
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 0 0]]
G_est = 
 [[-1.  1.  1.]
 [ 0. -1.  1.]
 [ 1.  0. -1.]]
Final Loss =  5.517327855736365
Final penalty =  2.4999999999518208
Final likelihood =  3.0173278557845444
rate =  0.15

SCM:  3
likelihood_true =  3.017964244356776
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
G_est = 
 [[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1. -1.]]
Final Loss =  31.81397510075992
Final penalty =  1.5000000000003813
Final likelihood =  30.31397510075954
rate =  0.07

